In [1]:
pip install spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 940.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58760 sha256=be9f410b57816c3bc8a0b29980ed28fbecef0914bc8535139ec747bcfad2797b
  Stored in directory: /root/.cache/pip/wheels/bc/0f/6c/b41528ca0fd4d46513185e90da91bc1c484bba6138641c5a62
Successfully built spark


In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=03c56163f2c5b47c153cd37023522060e455f3403e2e563f57c7cf6aad8611d4
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]"). appName("exercise").getOrCreate()

In [4]:
spark

In [5]:
# reading dataset 
from google.colab import files
uploaded = files.upload()

Saving Sample - EU Superstore.csv to Sample - EU Superstore.csv


In [6]:
df = spark.read.option('header','true') .csv("Sample - EU Superstore.csv")

In [7]:
df.printSchema()

root
 |-- Row ID: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: string (nullable = true)



In [8]:
df.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corporate|        Leeds|             England|United Kingdom|  North|OFF-ST-10000988|Office Supplies|     Storage|Fellowes 

In [9]:
df.count()

10000

In [10]:
# how many rows of the EU Superstore dataset have the country being France
from pyspark.sql.functions import col
france_df = df.filter(col('Country')=="France")

In [11]:
france_df.count()

2827

In [12]:
# of those, how many are profitable?
france_profitable_count = france_df.filter( col("Profit") > 0)

In [13]:
france_profitable_count.count()

2277

In [14]:
# how any different discount brackets exist? what are they?
discounts = df.select(col("Discount")).distinct()

In [15]:
discounts.show()

+--------+
|Discount|
+--------+
|     0.3|
|     0.7|
|       0|
|     0.2|
|    0.15|
|    0.35|
|     0.8|
|    0.45|
|     0.5|
|    0.65|
|     0.6|
|     0.1|
|    0.85|
|     0.4|
+--------+



In [16]:
discounts.count()

14

In [17]:
# let's see the totl profit by discount bracket, make sure they are ordered by 
from pyspark.sql.functions import col, sum
profit_by_discount = df.groupBy(col("Discount")).agg(sum(col("Profit")).alias("Total_Profit"))
profit_by_discount = profit_by_discount.orderBy(col("Discount"))

In [18]:
profit_by_discount.show()

+--------+-------------------+
|Discount|       Total_Profit|
+--------+-------------------+
|       0| 383806.53000000026|
|     0.1|  126884.0309999999|
|    0.15| 24677.563499999975|
|     0.2| 2189.5499999999984|
|     0.3| -758.4209999999999|
|    0.35|          -9122.649|
|     0.4|-21346.427999999996|
|    0.45|         -1103.1915|
|     0.5|         -96632.115|
|     0.6|-20517.456000000002|
|    0.65| -6221.965499999999|
|     0.7|          -5496.765|
|     0.8|           -460.284|
|    0.85|          -3068.658|
+--------+-------------------+



In [19]:
# what is the value after which we should stop offering discount?
from pyspark.sql.functions import when, lag 
profit_by_discount = profit_by_discount.withColumn('Profitability',
                                                   when(profit_by_discount['Total_Profit'] > 0,
                                                        'profitable')
                                                   .otherwise('nonprofitable')
                                                  )
profit_by_discount.show()

+--------+-------------------+-------------+
|Discount|       Total_Profit|Profitability|
+--------+-------------------+-------------+
|       0| 383806.53000000026|   profitable|
|     0.1|  126884.0309999999|   profitable|
|    0.15| 24677.563499999975|   profitable|
|     0.2| 2189.5499999999984|   profitable|
|     0.3| -758.4209999999999|nonprofitable|
|    0.35|          -9122.649|nonprofitable|
|     0.4|-21346.427999999996|nonprofitable|
|    0.45|         -1103.1915|nonprofitable|
|     0.5|         -96632.115|nonprofitable|
|     0.6|-20517.456000000002|nonprofitable|
|    0.65| -6221.965499999999|nonprofitable|
|     0.7|          -5496.765|nonprofitable|
|     0.8|           -460.284|nonprofitable|
|    0.85|          -3068.658|nonprofitable|
+--------+-------------------+-------------+



In [20]:
from pyspark.sql.window import Window

negative_profit_discount = (profit_by_discount.withColumn('Prev_Total_Profit', lag('Total_Profit')
  .over(Window.orderBy('Discount'))).filter((col('Total_Profit') < 0) & (col('Prev_Total_Profit') > 0))
                            .select('Discount').orderBy('Discount').first())
if negative_profit_discount:
    print(f"The first discount level after which the total profit becomes negative is: {negative_profit_discount[0]}")
else:
    print("All discount levels are profitable")

The first discount level after which the total profit becomes negative is: 0.3


In [21]:
# who are the top 5 most profitable customers
from pyspark.sql.functions import sum

customer_profit = df.groupBy("Customer ID", "Customer Name").agg(sum("Profit").alias("Total Profit"))
top_customers = customer_profit.orderBy("Total Profit", ascending=False).limit(5)

top_customers.show()



+-----------+-----------------+------------------+
|Customer ID|    Customer Name|      Total Profit|
+-----------+-----------------+------------------+
|   SP-20920|     Susan Pistek| 4974.512999999999|
|   PJ-18835|    Patrick Jones|3986.0039999999995|
|   PO-18865|Patrick O'Donnell|          3778.197|
|   EB-13840|    Ellis Ballard|           3459.66|
|   MG-18145|  Mike Gockenbach|3144.4439999999995|
+-----------+-----------------+------------------+



In [22]:
# get all the rows belonging to those 5 customer names: hint, you may need the collect method - how many rows are they?
customer_names = [row[1] for row in top_customers.collect()]
# Filter the original dataframe for rows belonging to these customers
rows = df.filter(col("Customer Name").isin(customer_names))

# Count the number of rows
print("Number of rows belonging to the top 5 most profitable customers:", rows.count())

rows.show()

Number of rows belonging to the top 5 most profitable customers: 76
+------+---------------+----------+----------+--------------+-----------+-----------------+---------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+-------+--------+--------+-------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|    Customer Name|  Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|  Sales|Quantity|Discount| Profit|
+------+---------------+----------+----------+--------------+-----------+-----------------+---------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+-------+--------+--------+-------+
|     8|ES-2015-5113958|02/08/2015|07/08/2015|  Second Class|   EB-13840|    Ellis Ballard|Corporate|West Bromwich|             England|United Kingdom| 

In [23]:
# create a new column which is the value of the sale were there not discount applied. Hint: orginal = sales/(1-d)

df = df.withColumn("Original_Sales", col("Sales") / (1 - col("Discount")))


In [24]:
df.select('Original_Sales', 'Sales', 'Discount').show()

+------------------+--------+--------+
|    Original_Sales|   Sales|Discount|
+------------------+--------+--------+
|              79.2|    79.2|       0|
|            388.92|  388.92|       0|
|             35.19|   35.19|       0|
|             50.94|   50.94|       0|
|            307.44|  307.44|       0|
|             122.4|   122.4|       0|
|            413.82|  413.82|       0|
|            428.22|  428.22|       0|
|           3979.29| 3979.29|       0|
|             43.56|   43.56|       0|
|             25.26|   25.26|       0|
|2715.4500000000003|2443.905|     0.1|
|             12.21|   12.21|       0|
|2549.7599999999998|2167.296|    0.15|
|            153.45| 138.105|     0.1|
|142.64999999999998| 128.385|     0.1|
|            690.12|  690.12|       0|
|              8.16|    8.16|       0|
|            347.88|  347.88|       0|
| 639.4499999999999| 575.505|     0.1|
+------------------+--------+--------+
only showing top 20 rows



In [25]:
# calculate the difference between sales and discount value
df = df.withColumn("Difference", col("Sales") - col("Original_Sales"))

# Show the resulting DataFrame
df.select("Sales", "Discount", "Original_Sales", "Difference").show()


+--------+--------+------------------+-------------------+
|   Sales|Discount|    Original_Sales|         Difference|
+--------+--------+------------------+-------------------+
|    79.2|       0|              79.2|                0.0|
|  388.92|       0|            388.92|                0.0|
|   35.19|       0|             35.19|                0.0|
|   50.94|       0|             50.94|                0.0|
|  307.44|       0|            307.44|                0.0|
|   122.4|       0|             122.4|                0.0|
|  413.82|       0|            413.82|                0.0|
|  428.22|       0|            428.22|                0.0|
| 3979.29|       0|           3979.29|                0.0|
|   43.56|       0|             43.56|                0.0|
|   25.26|       0|             25.26|                0.0|
|2443.905|     0.1|2715.4500000000003| -271.5450000000001|
|   12.21|       0|             12.21|                0.0|
|2167.296|    0.15|2549.7599999999998|-382.4639999999999

In [26]:
# how much money did we not gain due to the discounts - per discount bracket?

df = df.withColumn("Missed_Sale", col("Original_Sales") - col("Sales"))

# Group by discount bracket and sum the missed sale
missed_sales = df.groupBy("Discount").agg(sum("Missed_Sale").alias("Missed_Sales"))
missed_sales.show()

+--------+------------------+
|Discount|      Missed_Sales|
+--------+------------------+
|     0.3|2630.2410000000004|
|     0.7|          8534.085|
|       0|               0.0|
|     0.2|10653.119999999997|
|    0.15| 45233.17650000002|
|    0.35|29163.099000000002|
|     0.8| 635.6640000000002|
|    0.45|2083.4414999999995|
|     0.5|183734.26500000045|
|    0.65|12219.655499999999|
|     0.6| 39644.04599999997|
|     0.1| 84712.44899999995|
|    0.85|          4515.438|
|     0.4| 46724.68800000002|
+--------+------------------+



In [27]:
# find the discount bracket which made us not gain the most (dynamically)
from pyspark.sql.functions import desc
worst_discount = missed_sales.orderBy(desc("Missed_Sales"))
worst_discount.show(5)

+--------+------------------+
|Discount|      Missed_Sales|
+--------+------------------+
|     0.5|183734.26500000045|
|     0.1| 84712.44899999995|
|     0.4| 46724.68800000002|
|    0.15| 45233.17650000002|
|     0.6| 39644.04599999997|
+--------+------------------+
only showing top 5 rows



In [28]:
# what would have been the total profit if we removed all orders from that discount group? 
total_profit = df.agg(sum("Profit")).collect()[0][0]
print("Total Profit without applying any discount:", total_profit)


Total Profit without applying any discount: 372829.7415000005


In [29]:
#how much more (or less) profit is that?
# i didnt understand 

In [31]:
# create a temporary table for our superstore table in sql
df.createOrReplaceTempView("superstore_temp")

In [32]:
spark.sql("SELECT * FROM superstore_temp").show()


+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+------------------+-------------------+------------------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|    Original_Sales|         Difference|       Missed_Sale|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+------------------+-------------------+------------------+
|     1|ES-2017-1311038|07/02/2017|11/02

In [33]:
# use an SQL query to count the number of rows
count_rows = spark.sql("SELECT COUNT(*) FROM superstore_temp")
count_rows.show()

+--------+
|count(1)|
+--------+
|   10000|
+--------+



In [34]:
# Use an SQL query to calculate the profit ratio for each country: hint, ratio is sum(profit)/sum(sales)

profit_ratio_by_country = spark.sql("""
    SELECT `Country`,
           SUM(`Profit`) / SUM(`Sales`) AS `Profit_Ratio`
    FROM superstore_temp
    GROUP BY `Country`
    ORDER BY `Profit_Ratio` Desc
""")

# show the results
profit_ratio_by_country.show()


+--------------+--------------------+
|       Country|        Profit_Ratio|
+--------------+--------------------+
|   Switzerland|  0.2909201193350232|
|       Austria|  0.2641908775042505|
|        Norway|  0.2517747548521659|
|       Belgium| 0.23508766583987942|
|United Kingdom| 0.21170103540397134|
|         Spain| 0.18941580658358978|
|       Finland| 0.18864296633316185|
|       Germany| 0.17066792076621765|
|        France| 0.12693568221933804|
|         Italy| 0.06844355185424991|
|       Ireland|-0.44426677493909256|
|       Denmark| -0.4957190005664471|
|   Netherlands| -0.5298342790541865|
|        Sweden| -0.5745674280714466|
|      Portugal| -0.5761662270806188|
+--------------+--------------------+



In [35]:
# is the country with the largest profit ratio, the country with the largest profit?

# calculate profit by country
profit_by_country = df.groupBy("Country").agg(sum("Profit").alias("Total_profit_per_country"))

In [36]:
profit_by_country.orderBy("Total_profit_per_country", ascending=False).show()


+--------------+------------------------+
|       Country|Total_profit_per_country|
+--------------+------------------------+
|United Kingdom|      111900.15000000001|
|        France|      109029.00299999975|
|       Germany|      107322.82049999991|
|         Spain|       54390.11999999999|
|       Austria|                21442.26|
|         Italy|      19828.757999999965|
|       Belgium|                11572.59|
|   Switzerland|       7237.470000000001|
|        Norway|                 5167.77|
|       Finland|                 3905.73|
|       Denmark|     -4282.0470000000005|
|       Ireland|      -7392.381000000003|
|      Portugal|      -8703.059999999998|
|        Sweden|     -17519.366999999987|
|   Netherlands|      -41070.07499999996|
+--------------+------------------------+

